<font size="8">Data Acquisition, Modeling and Analysis: Deep Learning - Final Project</font>\
<font size="6">Written by Alexander M. Pellegrino</font>\
<font size="6">Under Dr. Rensheng Wang</font>\
<font size="6">On March 30th, 2024</font>

In [106]:
import io
import numpy as np
from PIL import Image
import tensorflow as tf
import ipywidgets as widgets
from keras.preprocessing import image
#from keras.src.saving.saving_api import load_model
from keras.src.applications.resnet_v2 import preprocess_input

# Load Quantized (Faster, Inference-Only) Model
interpreter = tf.lite.Interpreter(model_path='ResNet152V2_Cat_Dog_Classifier_Unquantized.tflite')
interpreter.allocate_tensors()

# To Use Raw Model
#model = load_model('ResNet152V2_Cat_Dog_Classifier_E20-val_acc0.9425.keras', compile=False)

<font size="4">Comment out the cell below if using the raw model instead of the quantized version.</font>

In [107]:
# Get I/O Information from TFLite Model
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Set Expected Input Dimensions
input_shape = input_details[0]['shape']

<font size="4">The following Jupyter Widgets make up the actual frontend of the program and are universal - keep this section as-is regardless of which model is being used.</font>

In [108]:
upload_box = widgets.FileUpload(
    accept='image/*',  # Only allow image files
    multiple=False     # Only allow one file at a time
)

button = widgets.Button(
    description='Process Image',
    button_style='info',
    tooltip='Click to process image.'
)

output = widgets.Output()

<font size="4">Two variants of process_image function are provided below. Currently, the TFLite model is being used, but to use a full keras model comment out the uncommented version and uncomment the commented out one. Note that the "b" parameter is required for the function to be used as a callback by the Button Widget - the underscore marks that it's an unused variable.</font>

In [109]:
# Use this version for TFLite (Quantized) Model
@output.capture(clear_output=True)
def process_image(_b):
    if not upload_box.value:
        print("No file uploaded.")
        return
    
    input_file = upload_box.value[0]
    print(f"Loading {input_file['name']}...")
    
    try:
        input_image = Image.open(io.BytesIO(input_file['content']))
        print(f"{input_file['name']} opened successfully.")
        
    except Exception as e:
        print(f"Failed to open image: {e}")
        return

    print("Preforming preprocessing on image...")
    try:
        input_image = input_image.resize((input_shape[1], input_shape[2]))
        image_array = image.img_to_array(input_image)
        preprocess_input(image_array)
        image_array = np.expand_dims(image_array, axis=0)
        print("Image preprocessing completed successfully.")
        
    except Exception as e:
        print(f"Error in image preprocessing: {e}")
        return

    print("Analyzing...")

    # Ensure the input type matches the model's input type
    image_array = image_array.astype(input_details[0]['dtype'])

    # Point the TFLite Input Tensors to the input data
    interpreter.set_tensor(input_details[0]['index'], image_array)

    # Run Inference
    interpreter.invoke()

    # Read Output Tensors
    output_data = interpreter.get_tensor(output_details[0]['index'])

    # Binary Classification Output
    prediction = output_data[0][0]
    print(prediction)
    if prediction > 0.5:
        print("You Uploaded a Dog")
    else:
        print("You Uploaded a Cat")

# Use this version for Raw Model
# @output.capture(clear_output=True)
# def process_image(_b):
#     if not upload_box.value:
#         print("No file uploaded.")
#         return
#     
#     input_file = next(iter(upload_box.value.values()))
#     
#     input_image = Image.open(input_file['content'])
#     input_image = input_image.resize((224, 224))
#     image_array = image.img_to_array(input_image)
#     image_array = np.expand_dims(image_array, axis=0)
#     
#     prediction = model.predict(image_array)
#     # Output the raw prediction or probability
#     if prediction > 0.5:
#         print("You Uploaded a Dog")
#     else:
#         print("You Uploaded a Cat")

In [110]:
button.on_click(process_image)

display(upload_box)
display(button)
display(output)

FileUpload(value=(), accept='image/*', description='Upload')

Button(button_style='info', description='Process Image', style=ButtonStyle(), tooltip='Click to process image.…

Output()